In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
data = pd.read_sql_query('SELECT * FROM loan', engine)
data.head()

In [ ]:
query = '''select * from trans t
left join loan l
on t.account_id = l.account_id
where l.status in ('A', 'B');'''

data = pd.read_sql_query(query, engine)
data.head()

In [ ]:
#more specific with our fields 

# Extracting the data (the previous query modified)

query = '''select t.type, t.operation, t.amount as t_amount, t.balance, t.k_symbol, l.amount as l_amount, l.duration, l.payments, l.status
from trans t
left join loan l
on t.account_id = l.account_id
where l.status in ('A', 'B');'''
data = pd.read_sql_query(query, engine)
data.head()

In [ ]:
data.shape

In [ ]:

data.dtypes

In [ ]:
data['duration'] = data['duration'].astype('object') # This will be treated as categorical
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
## checking all the categorical columns
data['operation'].value_counts()


In [ ]:
def cleanOperation(x):
    x = x.lower()
    if 'vyber' in x:
        return "vyber"
    elif 'prevod' in x:
        return "prevod"
    elif 'vklad' in x:
        return 'vklad'
    else:
        return 'unknown'

data['operation'] = list(map(cleanOperation, data['operation']))

In [ ]:
data['operation'].value_counts()

In [ ]:
data['k_symbol'].value_counts()

In [ ]:
data['k_symbol'].value_counts().index

In [ ]:
def cleankSymbol(x):
    if x in ['', ' ']:
        return 'unknown'
    else:
        return x

data['k_symbol'] = list(map(cleankSymbol, data['k_symbol']))

In [ ]:
data['k_symbol'].value_counts()

In [ ]:
data = data[~data['k_symbol'].isin(['POJISTINE', 'SANKC. UROK', 'UVER'])]

In [ ]:
data['k_symbol'].value_counts()

In [ ]:
data['duration'].value_counts().index

In [ ]:
def cleanDuration(x):
    if x in [48, 60]:
        return 'other'
    else:
        return str(x)
data['duration'] = list(map(cleanDuration, data['duration']))
data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Checking for multicollinearity

corr_matrix=data.corr(method='pearson')  # default
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
#sns.distplot(data['t_amount'])
#plt.show()

#sns.distplot(data['l_amount'])
#plt.show()

sns.distplot(data['balance'])
plt.show()

#sns.distplot(data['payments'])
#plt.show()

In [ ]:
from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import StandardScaler

In [ ]:
import numpy as np 
X = data.select_dtypes(include = np.number)
# Normalizing data
transformer = Normalizer().fit(X)
x_normalized = transformer.transform(X)
x = pd.DataFrame(x_normalized)

In [ ]:
sns.distplot(data['balance'])
plt.show()

In [ ]:
cat = data.select_dtypes(include = np.object)
cat = cat.drop(['status'], axis=1)
categorical = pd.get_dummies(cat, columns=['type', 'operation', 'k_symbol', 'duration'])

In [ ]:
categorical.head()

In [ ]:
y = data['status']
X = np.concatenate((x, categorical), axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

In [ ]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)
# this gives me an error because of the scaling of the data
# I am feeding into the model however the model still runs 
# so I have not fixed it yet! data requires more pre processing

In [ ]:
classification.score(X_test, y_test)
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

In [ ]:
print(y_test.value_counts())
# As you will notice here, there is a huge imbalance in the data among the different classes. We will talk more about imbalance and how to resolve it later

In [ ]:
pd.Series(predictions).value_counts()
# This shows that the disparity in the numbers are amplified by the model

In [ ]:
#lets bring in the confusion matrix

from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, predictions)
print(cf_matrix)

The default correlation matrix array is ugly - I found this article to walk through how to make it prettier 
https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea

In [ ]:
sns.heatmap(cf_matrix, annot=True)

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

In [ ]:
# time for the ROC and AUC analysis 
#- for this to work the expectation is that the predictor is either 1 or 0 
# in my case i have binary but its A or B 
#so I am arbitrarily assigning B as the true/false question
#ie, if its meant to be B, and i predict B then true positive etc 

import sklearn.metrics as metrics
import matplotlib.pyplot as pyplt

y_pred_proba = classification.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba, pos_label='B')
pyplt.plot(fpr,tpr)

In [ ]:
auc = metrics.roc_auc_score(y_test, y_pred_proba)
print(auc)

In general, an AUC of 0.5 suggests no discrimination (i.e., ability to diagnose patients with and without the disease or condition based on the test), 0.7 to 0.8 is considered acceptable, 0.8 to 0.9 is considered excellent, and more than 0.9 is considered outstanding.